# Projeto AM - Leandro
## XGBoost, CatBoost, and LightGBM

In [1]:
import traceback
import pandas as pd
import warnings
from pathlib import Path
from data.data import DataProcessor
from evaluation.evaluation import ResultsManager
from experiments.experiment_boosting import BoostingExperiment
from experiments.experiment_boosting_gpu import BoostingExperimentGPU

## Configuração dos Experimentos

In [9]:
CACHE_DIR = "./data/openml_cache"

SEED = 123
N_TRIALS = 50  # Optuna trials
CV_FOLDS = 10  # Cross-validation folds

# Modelos a serem testados
MODELS = ["xgboost", "catboost", "lightgbm"]

## Função para agregar resultados

In [ ]:
def aggregate_results(results_dir: str, model_name: str, output_filename: str = None):
    """
    Agrega todos os resultados individuais em um único CSV

    Args:
        results_dir: Diretório com os resultados (ex: "./results/catboost_gpu")
        model_name: Nome do modelo (ex: "catboost")
        output_filename: Nome do arquivo de saída (opcional, default: {model_name}_metrics.csv)

    Returns:
        DataFrame com todos os resultados agregados
    """
    results_path = Path(results_dir)
    individual_dir = results_path / "individual_results"

    # Verificar se diretório existe
    if not individual_dir.exists():
        print(f"❌ Diretório não encontrado: {individual_dir}")
        return None

    # Encontrar todos os arquivos de resultado
    result_files = list(individual_dir.glob("*_result.csv"))

    if not result_files:
        print(f"❌ Nenhum arquivo de resultado encontrado em: {individual_dir}")
        return None

    print(f"✓ Encontrados {len(result_files)} arquivos de resultado")
    print(f"  Diretório: {individual_dir}\n")

    # Ler todos os resultados
    all_results = []

    for result_file in result_files:
        try:
            df = pd.read_csv(result_file)
            all_results.append(df)
            dataset_name = result_file.stem.replace("_result", "")
            print(f"  ✓ {dataset_name}")
        except Exception as e:
            print(f"  ✗ Erro ao ler {result_file.name}: {e}")

    if not all_results:
        print("\n❌ Nenhum resultado foi carregado com sucesso")
        return None

    # Concatenar todos os resultados
    df_all = pd.concat(all_results, ignore_index=True)

    # Ordenar por dataset
    df_all = df_all.sort_values("dataset").reset_index(drop=True)

    # Selecionar colunas importantes para o CSV de métricas
    metric_columns = [
        "dataset",
        "model",
        "accuracy",
        "auc_ovo",
        "gmean",
        "cross_entropy",
    ]

    # Adicionar colunas opcionais se existirem
    optional_columns = [
        "n_samples_train",
        "n_samples_test",
        "n_features",
        "n_classes",
        "tuning_time",
        "training_time",
        "prediction_time",
        "total_time",
    ]

    for col in optional_columns:
        if col in df_all.columns:
            metric_columns.append(col)

    # Filtrar apenas as colunas que existem
    metric_columns = [col for col in metric_columns if col in df_all.columns]

    df_metrics = df_all[metric_columns].copy()

    # Definir nome do arquivo de saída
    if output_filename is None:
        output_filename = f"{model_name}_metrics.csv"

    output_path = results_path / output_filename

    # Salvar CSV
    df_metrics.to_csv(output_path, index=False)

    print(f"\n{'=' * 80}")
    print("RESULTADOS AGREGADOS")
    print(f"{'=' * 80}")
    print(f"✓ Total de datasets: {len(df_metrics)}")
    print(f"✓ Arquivo salvo: {output_path}")
    print(f"✓ Colunas: {list(df_metrics.columns)}")
    print(f"{'=' * 80}\n")

    # Mostrar estatísticas
    print("Estatísticas das Métricas:")
    print(df_metrics[["accuracy", "auc_ovo", "gmean", "cross_entropy"]].describe())

    print(f"\n{'=' * 80}")
    print("DATASETS PROCESSADOS (ordenados alfabeticamente):")
    print(f"{'=' * 80}")
    for idx, dataset in enumerate(df_metrics["dataset"].sort_values(), 1):
        print(f"{idx:3d}. {dataset}")
    print(f"{'=' * 80}\n")

    return df_metrics

## Função Principal de Execução

In [ ]:
def run_experiments_for_model(
    model_name: str,
    use_gpu: bool = False,
    gpu_id: int = 0,
    n_trials: int = N_TRIALS,
    cv_folds: int = CV_FOLDS,
    seed: int = SEED,
    cache_dir: str = "./data/openml_cache",
):
    """
    Executa experimentos para um modelo específico em todos os datasets

    Args:
        model_name: Nome do modelo ('xgboost', 'catboost', ou 'lightgbm')
        use_gpu: Se True, usa BoostingExperimentGPU. Se False, usa BoostingExperiment
        gpu_id: ID da GPU a ser usada (0, 1, 2, ...)
        n_trials: Número de trials do Optuna (padrão: 50)
        cv_folds: Número de folds de validação cruzada (padrão: 10)
        seed: Seed
        cache_dir: Diretório com datasets processados
    """
    # Configurar diretório de resultados
    if use_gpu:
        RESULTS_DIR = f"./results/{model_name}_gpu"
    else:
        RESULTS_DIR = f"./results/{model_name}"

    print("=" * 80)
    print(f"{model_name.upper()} - EXPERIMENT PIPELINE")
    print("=" * 80)
    print(f"Model: {model_name}")
    print(f"Mode: {'GPU' if use_gpu else 'CPU'}")
    if use_gpu:
        print(f"GPU ID: {gpu_id}")
    print(f"Seed: {seed}")
    print(f"Optuna trials: {n_trials}")
    print(f"CV folds: {cv_folds}")
    print(f"Results directory: {RESULTS_DIR}")
    print("=" * 80)

    # Initialize data processor and results manager
    processor = DataProcessor(seed=seed, cache_dir=cache_dir)
    results_manager = ResultsManager(save_dir=RESULTS_DIR, model_name=model_name)

    # Find all processed PKL files
    pkl_files = list(Path(cache_dir).glob("*_dataset.pkl"))

    if not pkl_files:
        print("\nNo processed datasets found!")
        print(f"Looking in: {cache_dir}")
        print("\nPlease run data.py first to download and process datasets.")
        return

    print(f"\n✓ Found {len(pkl_files)} processed datasets\n")

    # Check GPU availability if requested
    if use_gpu:
        try:
            import torch

            if torch.cuda.is_available():
                print(f"✓ GPU available: {torch.cuda.get_device_name(gpu_id)}")
                print(
                    f"  Memory: {torch.cuda.get_device_properties(gpu_id).total_memory / 1e9:.2f} GB\n"
                )
            else:
                print("Warning: GPU requested but not available, using CPU\n")
                use_gpu = False
        except ImportError:
            print("PyTorch not installed, cannot check GPU")

    # Run experiments on all datasets
    failed_datasets = []

    for idx, pkl_file in enumerate(pkl_files[18:], 1):
        # Extract dataset name
        dataset_name = pkl_file.stem.replace("_dataset", "")

        print(f"\n{'#' * 80}")
        print(f"DATASET {idx}/{len(pkl_files)}: {dataset_name}")
        print(f"{'#' * 80}")

        try:
            # Load processed dataset (numpy arrays)
            X_train, y_train, X_test, y_test = processor.load_or_process_dataset(
                name=dataset_name
            )

            # Get number of classes
            n_classes = len(set(y_train))

            # Create experiment (GPU ou CPU)
            if use_gpu:
                # Usar versão GPU
                experiment = BoostingExperimentGPU(
                    dataset_name=dataset_name,
                    X_train=X_train,
                    y_train=y_train,
                    X_test=X_test,
                    y_test=y_test,
                    n_classes=n_classes,
                    model_type=model_name,
                    n_trials=n_trials,
                    cv_folds=cv_folds,
                    seed=seed,
                    use_gpu=True,
                    gpu_id=gpu_id,
                )
            else:
                # Usar versão CPU
                experiment = BoostingExperiment(
                    dataset_name=dataset_name,
                    X_train=X_train,
                    y_train=y_train,
                    X_test=X_test,
                    y_test=y_test,
                    n_classes=n_classes,
                    model_type=model_name,
                    n_trials=n_trials,
                    cv_folds=cv_folds,
                    seed=seed,
                    catboost_bootstrap_type="Bayesian",
                )

            # Run complete experiment
            results = experiment.run_complete_experiment()

            # Extract metrics and info for saving
            metrics = {
                "accuracy": results["accuracy"],
                "auc_ovo": results["auc_ovo"],
                "gmean": results["gmean"],
                "cross_entropy": results["cross_entropy"],
            }

            dataset_info = {
                "n_samples_train": results["n_samples_train"],
                "n_samples_test": results["n_samples_test"],
                "n_features": results["n_features"],
                "n_classes": results["n_classes"],
            }

            timings = {
                "tuning_time": results["tuning_time"],
                "training_time": results["training_time"],
                "prediction_time": results["prediction_time"],
                "total_time": results["total_time"],
            }

            # Adicionar info de GPU se disponível
            if use_gpu and "gpu_enabled" in results:
                timings["gpu_enabled"] = results["gpu_enabled"]

            # Save results
            results_manager.save_dataset_result(
                dataset_name=dataset_name,
                metrics=metrics,
                dataset_info=dataset_info,
                timings=timings,
                hyperparameters=results["best_params"],
            )

            print(f"\n✓ Completed {dataset_name} ({idx}/{len(pkl_files)})")

        except Exception as e:
            print(f"\n✗ Failed: {dataset_name}")
            print(f"   Error: {e}")
            failed_datasets.append((dataset_name, e))
            traceback.print_exc()
            continue

    # ========================================================================
    # Save final results
    # ========================================================================
    print("\n" + "=" * 80)
    print("SAVING FINAL RESULTS")
    print("=" * 80)

    # Save metrics CSV (for hypothesis testing)
    results_manager.save_metrics_csv()

    # Print final summary
    results_manager.print_summary()

    # Print failed datasets if any
    if failed_datasets:
        print(f"\nFailed datasets ({len(failed_datasets)}):")
        for dataset, error in failed_datasets:
            print(f"  - {dataset}: {error}")

    print("\n" + "=" * 80)
    print(f"EXPERIMENT COMPLETE FOR {model_name.upper()}!")
    print("=" * 80)
    print(f"✓ Total datasets: {len(pkl_files)}")
    print(f"✓ Successful: {len(results_manager.all_results)}")
    print(f"✓ Failed: {len(failed_datasets)}")
    print(f"\n✓ Metrics CSV saved: {RESULTS_DIR}/{model_name}_metrics.csv")
    print("=" * 80)

## Experimentos com XGBoost

In [ ]:
run_experiments_for_model("xgboost")

In [6]:
aggregate_results(results_dir="./results/xgboost", model_name="xgboost")

✓ Encontrados 30 arquivos de resultado
  Diretório: results\xgboost\individual_results

  ✓ analcatdata_authorship
  ✓ analcatdata_dmft
  ✓ balance-scale
  ✓ banknote-authentication
  ✓ blood-transfusion-service-center
  ✓ breast-w
  ✓ car
  ✓ climate-model-simulation-crashes
  ✓ cmc
  ✓ cnae-9
  ✓ credit-approval
  ✓ credit-g
  ✓ cylinder-bands
  ✓ diabetes
  ✓ dresses-sales
  ✓ eucalyptus
  ✓ ilpd
  ✓ kc2
  ✓ mfeat-factors
  ✓ MiceProtein
  ✓ pc1
  ✓ pc3
  ✓ pc4
  ✓ qsar-biodeg
  ✓ semeion
  ✓ steel-plates-fault
  ✓ tic-tac-toe
  ✓ vehicle
  ✓ vowel
  ✓ wdbc

RESULTADOS AGREGADOS
✓ Total de datasets: 30
✓ Arquivo salvo: results\xgboost\xgboost_metrics.csv
✓ Colunas: ['dataset', 'model', 'accuracy', 'auc_ovo', 'gmean', 'cross_entropy', 'n_samples_train', 'n_samples_test', 'n_features', 'n_classes', 'tuning_time', 'training_time', 'prediction_time', 'total_time']

Estatísticas das Métricas:
        accuracy    auc_ovo      gmean  cross_entropy
count  30.000000  30.000000  30.000000    

,dataset,model,accuracy,auc_ovo,gmean,cross_entropy,n_samples_train,n_samples_test,n_features,n_classes,tuning_time,training_time,prediction_time,total_time
0,MiceProtein,xgboost,0.972222,0.999671,0.971667,0.180083,756,324,77,8,1157.593033,3.263902,0.008860,1160.865796
1,analcatdata_authorship,xgboost,0.980237,0.997829,0.959238,0.069780,588,253,70,4,1.469057,0.137714,0.003011,1.609781
2,analcatdata_dmft,xgboost,0.236181,0.551220,0.231849,1.698971,462,199,7,5,175.707135,0.207949,0.006982,175.922066
3,balance-scale,xgboost,0.909574,0.960476,0.628629,0.217733,437,188,4,3,121.686057,0.272283,0.003989,121.962329
4,banknote-authentication,xgboost,0.995146,0.999976,0.995084,0.030669,960,412,4,2,64.321503,0.103219,0.001995,64.426717
5,blood-transfusion-service-center,xgboost,0.804444,0.746968,0.627332,0.486186,523,225,4,2,49.850096,0.072315,0.003990,49.926400
6,breast-w,xgboost,0.938095,0.979217,0.936275,0.160601,489,210,9,2,54.327947,0.092259,0.001995,54.422201
7,car,xgboost,0.967245,0.996231,0.904439,0.101350,1209,519,21,4,176.548280,0.253339,0.007979,176.809598
8,climate-model-simulation-crashes,xgboost,0.925926,0.884170,0.530899,0.229605,378,162,18,2,42.255156,0.095745,0.000997,42.351899
9,cmc,xgboost,0.518100,0.711147,0.495561,0.953966,1031,442,9,3,150.862976,0.385980,0.003990,151.252947


## Experimentos com CatBoost

In [5]:
run_experiments_for_model("catboost", use_gpu=True)

[I 2025-11-29 00:47:03,371] A new study created in memory with name: no-name-0a7dfa76-5156-46d4-8b90-84759eb30d1c


CATBOOST - EXPERIMENT PIPELINE
Model: catboost
Mode: GPU
GPU ID: 0
Seed: 123
Optuna trials: 50
CV folds: 10
Results directory: ./results/catboost_gpu

✓ Found 30 processed datasets

✓ GPU available: NVIDIA GeForce RTX 3060
  Memory: 12.88 GB


################################################################################
DATASET 1/30: analcatdata_authorship
################################################################################
Loading processed dataset from cache: analcatdata_authorship
Dataset: analcatdata_authorship
  Train: (588, 70), Test: (253, 70)
  Features: 70, Classes: 4
  Model: CATBOOST
  GPU: Enabled (Device 0)
  Bootstrap: Bayesian (fastest)
  Early Stopping: Enabled

EXPERIMENT: analcatdata_authorship

Starting hyperparameter optimization...
  Trials: 50, CV Folds: 10
  GPU: Enabled

✓ Optimization complete! Time: 1342.31s
  Best CV G-Mean: 0.9873
  Best parameters:
    iterations: 94
    depth: 3
    learning_rate: 0.28977739923128126
    l2_leaf_reg: 1.07255

[W 2025-11-29 10:30:12,640] Trial 46 failed with parameters: {'iterations': 499, 'depth': 10, 'learning_rate': 0.12908589029057904, 'l2_leaf_reg': 1.8665745998100671, 'bagging_temperature': 0.9089992939961165, 'min_data_in_leaf': 6} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "c:\Users\lucas\Desktop\mestrado\ml\projeto-leandro\venv_ml\Lib\site-packages\optuna\study\_optimize.py", line 205, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "c:\Users\lucas\Desktop\mestrado\ml\projeto-leandro\codigo gabriel\eu\experiment_boosting_gpu_optimized.py", line 295, in objective
    fold_model = self.train_model(
                 ^^^^^^^^^^^^^^^^^
  File "c:\Users\lucas\Desktop\mestrado\ml\projeto-leandro\codigo gabriel\eu\experiment_boosting_gpu_optimized.py", line 251, in train_model
    model.fit(
  File "c:\Users\lucas\Desktop\mestrado\ml\projeto-leandro\venv_ml\Lib\site-packages\catboost\core.py", lin

KeyboardInterrupt: 

In [5]:
run_experiments_for_model("catboost", use_gpu=True)

[I 2025-11-29 13:42:13,616] A new study created in memory with name: no-name-88c7fd5e-b9aa-4757-802d-e5b95c3e3c21


CATBOOST - EXPERIMENT PIPELINE
Model: catboost
Mode: GPU
GPU ID: 0
Seed: 123
Optuna trials: 50
CV folds: 10
Results directory: ./results/catboost_gpu

✓ Found 30 processed datasets

✓ GPU available: NVIDIA GeForce RTX 3060
  Memory: 12.88 GB


################################################################################
DATASET 1/30: vowel
################################################################################
Loading processed dataset from cache: vowel
Dataset: vowel
  Train: (693, 27), Test: (297, 27)
  Features: 27, Classes: 11
  Model: CATBOOST
  GPU: Enabled (Device 0)
  Bootstrap: Bayesian (fastest)
  Early Stopping: Enabled

EXPERIMENT: vowel

Starting hyperparameter optimization...
  Trials: 50, CV Folds: 10
  GPU: Enabled

✓ Optimization complete! Time: 4468.93s
  Best CV G-Mean: 0.9544
  Best parameters:
    iterations: 483
    depth: 10
    learning_rate: 0.26707484379625845
    l2_leaf_reg: 1.4213818814607038
    bagging_temperature: 0.792649414705147
    min_da

In [7]:
aggregate_results(results_dir="./results/catboost_gpu", model_name="catboost")

✓ Encontrados 30 arquivos de resultado
  Diretório: results\catboost_gpu\individual_results

  ✓ analcatdata_authorship
  ✓ analcatdata_dmft
  ✓ balance-scale
  ✓ banknote-authentication
  ✓ blood-transfusion-service-center
  ✓ breast-w
  ✓ car
  ✓ climate-model-simulation-crashes
  ✓ cmc
  ✓ cnae-9
  ✓ credit-approval
  ✓ credit-g
  ✓ cylinder-bands
  ✓ diabetes
  ✓ dresses-sales
  ✓ eucalyptus
  ✓ ilpd
  ✓ kc2
  ✓ mfeat-factors
  ✓ MiceProtein
  ✓ pc1
  ✓ pc3
  ✓ pc4
  ✓ qsar-biodeg
  ✓ semeion
  ✓ steel-plates-fault
  ✓ tic-tac-toe
  ✓ vehicle
  ✓ vowel
  ✓ wdbc

RESULTADOS AGREGADOS
✓ Total de datasets: 30
✓ Arquivo salvo: results\catboost_gpu\catboost_metrics.csv
✓ Colunas: ['dataset', 'model', 'accuracy', 'auc_ovo', 'gmean', 'cross_entropy', 'n_samples_train', 'n_samples_test', 'n_features', 'n_classes', 'tuning_time', 'training_time', 'prediction_time', 'total_time']

Estatísticas das Métricas:
        accuracy    auc_ovo      gmean  cross_entropy
count  30.000000  30.000000  30

,dataset,model,accuracy,auc_ovo,gmean,cross_entropy,n_samples_train,n_samples_test,n_features,n_classes,tuning_time,training_time,prediction_time,total_time
0,MiceProtein,catboost,0.981481,0.999923,0.981613,0.087627,756,324,77,8,3349.165567,4.883476,0.001964,3354.051007
1,analcatdata_authorship,catboost,0.976285,0.991943,0.958738,0.075878,588,253,70,4,1342.305657,0.806375,0.000000,1343.112032
2,analcatdata_dmft,catboost,0.221106,0.539319,0.202861,1.605199,462,199,7,5,612.767336,0.939453,0.000998,613.707788
3,balance-scale,catboost,0.904255,0.946322,0.500987,0.262813,437,188,4,3,897.549063,1.593031,0.000000,899.142094
4,banknote-authentication,catboost,0.997573,0.999952,0.997264,0.009123,960,412,4,2,1328.160801,4.393395,0.000968,1332.555165
5,blood-transfusion-service-center,catboost,0.791111,0.744369,0.584990,0.481412,523,225,4,2,620.634992,0.795215,0.000998,621.431205
6,breast-w,catboost,0.942857,0.983092,0.939869,0.141368,489,210,9,2,765.195982,1.648280,0.000997,766.845259
7,car,catboost,0.980732,0.999808,0.981696,0.085407,1209,519,21,4,1985.252568,1.829828,0.003989,1987.086385
8,climate-model-simulation-crashes,catboost,0.919753,0.867278,0.459772,0.216901,378,162,18,2,1009.208080,1.593813,0.000000,1010.801893
9,cmc,catboost,0.527149,0.700388,0.512370,0.977350,1031,442,9,3,1508.884505,1.667368,0.000966,1510.552839


## Experimentos com LightGBM

In [ ]:
warnings.filterwarnings(
    "ignore", category=UserWarning, message=".*does not have valid feature names.*"
)

In [16]:
run_experiments_for_model("lightgbm")

LIGHTGBM - EXPERIMENT PIPELINE
Model: lightgbm
Mode: CPU
Seed: 123
Optuna trials: 50
CV folds: 10
Results directory: ./results/lightgbm

✓ Found 30 processed datasets


################################################################################
DATASET 1/30: mfeat-factors
################################################################################
Loading processed dataset from cache: mfeat-factors
Dataset: mfeat-factors
  Train: (1400, 216), Test: (600, 216)
  Features: 216, Classes: 10
  Model: LIGHTGBM

EXPERIMENT: mfeat-factors

Starting hyperparameter optimization...
  Trials: 50, CV Folds: 10

✓ Optimization complete! Time: 1392.20s
  Best CV G-Mean: 0.9762
  Best parameters:
    n_estimators: 493
    max_depth: 3
    learning_rate: 0.035830515983720015
    subsample: 0.9785227828248506
    colsample_bytree: 0.9952433957260511
    min_child_samples: 97
    reg_alpha: 0.00383814423375467
    reg_lambda: 0.8357035402441148
    num_leaves: 26

Training final model...
✓ Trai

In [19]:
aggregate_results(results_dir="./results/lightgbm_final", model_name="lightgbm")

✓ Encontrados 30 arquivos de resultado
  Diretório: results\lightgbm_final\individual_results

  ✓ analcatdata_authorship
  ✓ analcatdata_dmft
  ✓ balance-scale
  ✓ banknote-authentication
  ✓ blood-transfusion-service-center
  ✓ breast-w
  ✓ car
  ✓ climate-model-simulation-crashes
  ✓ cmc
  ✓ cnae-9
  ✓ credit-approval
  ✓ credit-g
  ✓ cylinder-bands
  ✓ diabetes
  ✓ dresses-sales
  ✓ eucalyptus
  ✓ ilpd
  ✓ kc2
  ✓ mfeat-factors
  ✓ MiceProtein
  ✓ pc1
  ✓ pc3
  ✓ pc4
  ✓ qsar-biodeg
  ✓ semeion
  ✓ steel-plates-fault
  ✓ tic-tac-toe
  ✓ vehicle
  ✓ vowel
  ✓ wdbc

RESULTADOS AGREGADOS
✓ Total de datasets: 30
✓ Arquivo salvo: results\lightgbm_final\lightgbm_metrics.csv
✓ Colunas: ['dataset', 'model', 'accuracy', 'auc_ovo', 'gmean', 'cross_entropy', 'n_samples_train', 'n_samples_test', 'n_features', 'n_classes', 'tuning_time', 'training_time', 'prediction_time', 'total_time']

Estatísticas das Métricas:
        accuracy    auc_ovo      gmean  cross_entropy
count  30.000000  30.000000

,dataset,model,accuracy,auc_ovo,gmean,cross_entropy,n_samples_train,n_samples_test,n_features,n_classes,tuning_time,training_time,prediction_time,total_time
0,MiceProtein,lightgbm,0.972222,0.999713,0.971667,0.081560,756,324,77,8,545.476862,0.481711,0.010971,545.969544
1,analcatdata_authorship,lightgbm,0.980237,0.998959,0.959238,0.056321,588,253,70,4,562.540354,0.933503,0.001995,563.475853
2,analcatdata_dmft,lightgbm,0.216080,0.575025,0.210395,1.598736,462,199,7,5,408.131138,0.455782,0.000998,408.587918
3,balance-scale,lightgbm,0.920213,0.961344,0.633699,0.202047,437,188,4,3,434.957399,0.976389,0.002993,435.936780
4,banknote-authentication,lightgbm,1.000000,1.000000,1.000000,0.008070,960,412,4,2,250.728616,0.340091,0.001995,251.070701
5,blood-transfusion-service-center,lightgbm,0.782222,0.765053,0.495595,0.467822,523,225,4,2,130.466971,0.198468,0.000998,130.666437
6,breast-w,lightgbm,0.933333,0.983243,0.929261,0.154893,489,210,9,2,108.012728,0.164560,0.000000,108.177288
7,car,lightgbm,0.980732,0.999557,0.991576,0.068121,1209,519,21,4,944.546756,3.689135,0.015958,948.251848
8,climate-model-simulation-crashes,lightgbm,0.919753,0.879826,0.589483,0.223970,378,162,18,2,149.796075,0.425861,0.001995,150.223931
9,cmc,lightgbm,0.522624,0.709721,0.488263,0.935712,1031,442,9,3,628.163919,0.881642,0.001995,629.047556


## Resnet

(apenas agregando os resultados dos datasets individuais pra incluir o tempo)

In [4]:
aggregate_results(results_dir="./results/resnet", model_name="resnet")

✓ Encontrados 30 arquivos de resultado
  Diretório: results\resnet\individual_results

  ✓ analcatdata_authorship
  ✓ analcatdata_dmft
  ✓ balance-scale
  ✓ banknote-authentication
  ✓ blood-transfusion-service-center
  ✓ breast-w
  ✓ car
  ✓ climate-model-simulation-crashes
  ✓ cmc
  ✓ cnae-9
  ✓ credit-approval
  ✓ credit-g
  ✓ cylinder-bands
  ✓ diabetes
  ✓ dresses-sales
  ✓ eucalyptus
  ✓ ilpd
  ✓ kc2
  ✓ mfeat-factors
  ✓ MiceProtein
  ✓ pc1
  ✓ pc3
  ✓ pc4
  ✓ qsar-biodeg
  ✓ semeion
  ✓ steel-plates-fault
  ✓ tic-tac-toe
  ✓ vehicle
  ✓ vowel
  ✓ wdbc

RESULTADOS AGREGADOS
✓ Total de datasets: 30
✓ Arquivo salvo: results\resnet\resnet_metrics.csv
✓ Colunas: ['dataset', 'model', 'accuracy', 'auc_ovo', 'gmean', 'cross_entropy', 'n_samples_train', 'n_samples_test', 'n_features', 'n_classes', 'tuning_time', 'training_time', 'prediction_time', 'total_time']

Estatísticas das Métricas:
        accuracy    auc_ovo      gmean  cross_entropy
count  30.000000  30.000000  30.000000      3

,dataset,model,accuracy,auc_ovo,gmean,cross_entropy,n_samples_train,n_samples_test,n_features,n_classes,tuning_time,training_time,prediction_time,total_time
0,MiceProtein,resnet,1.000000,1.000000,1.000000,0.015068,756,324,77,8,5354.079092,16.843500,0.011285,5370.933877
1,analcatdata_authorship,resnet,0.988142,0.999905,0.991464,0.045285,588,253,70,4,4799.100377,13.060215,0.002005,4812.162596
2,analcatdata_dmft,resnet,0.231156,0.556069,0.227015,1.753764,462,199,7,5,420.625941,0.751168,0.002909,421.380017
3,balance-scale,resnet,0.978723,0.996818,0.965945,0.070676,437,188,4,3,503.645045,0.174086,0.000485,503.819616
4,banknote-authentication,resnet,1.000000,1.000000,1.000000,0.017180,960,412,4,2,2413.925925,1.024395,0.001544,2414.951865
5,blood-transfusion-service-center,resnet,0.604444,0.717349,0.653873,0.680959,523,225,4,2,824.185309,2.923539,0.002396,827.111244
6,breast-w,resnet,0.933333,0.983696,0.932667,0.319321,489,210,9,2,713.480148,1.936967,0.001030,715.418145
7,car,resnet,0.988439,0.999831,0.971309,0.027829,1209,519,21,4,3397.871263,11.452116,0.001740,3409.325118
8,climate-model-simulation-crashes,resnet,0.895062,0.910232,0.843439,0.313689,378,162,18,2,467.067612,0.352445,0.001011,467.421067
9,cmc,resnet,0.538462,0.719239,0.528419,0.984051,1031,442,9,3,924.599920,0.898741,0.005188,925.503849
